In [1]:
%pip install faiss-cpu
%pip install FlagEmbedding peft

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv

# 1. .env 파일 로드 (현재 디렉토리의 .env 파일을 찾음)
load_dotenv(override=True)

# 2. 환경 변수 가져오기
api_key = os.getenv("GEMINI_API_KEY")

In [2]:
import google.generativeai as genai

for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
      print(m.name)

/opt/anaconda3/envs/ai-engineering/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/envs/ai-engineering/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/h2/b2m866b95xlbn75zd0f36wq80000gn/T/ipykernel_46569/2872679221.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as pos

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


In [7]:
def embedding(texts) :
  return genai.embed_content(
    model="models/text-embedding-004",
    content=texts
  )["embedding"]

In [12]:
texts = ["This is a Test."]

embeds = embedding(texts)
print(embeds)
print(len(embeds[0]))

[[0.021709729, -0.010004897, -0.07831449, 0.00021625146, 0.016366882, -0.0061589633, 0.056820635, 0.03478271, -0.0053757853, 0.035010446, -4.1074087e-05, 0.01616381, 0.046615493, -0.01919706, -0.0007648253, -0.019785572, 0.026104966, 0.067962535, -0.06785214, -0.031471316, 0.020794155, -0.04611739, 0.0019919872, -0.035202015, -0.026550822, -0.033206027, 0.008064024, 0.003174477, 0.025845889, -0.021457084, 0.012842088, 0.043763965, 0.031452913, -0.004718091, 0.012690874, -0.011095253, -0.012762027, 0.020994188, 0.020329844, -0.080613144, 0.0061794934, 0.08040166, -0.076733366, -0.00773825, -0.025886035, -0.041227743, 0.038715918, 0.0044989176, 0.003333499, -0.0023913558, 0.0407256, 0.051695146, -0.057483457, -0.0028400905, 0.015980821, -0.009783751, 0.013114645, -0.046905294, 0.052408155, 0.0075120507, 0.041234136, -0.02858687, -0.0052159308, 0.005603428, 0.013024177, -0.0059801494, 0.014997179, -0.019358763, -0.03909497, -0.029858405, -0.071830705, 0.018266415, 0.0065697925, -0.0085571

In [13]:
in_texts = [
  "I'm glad it didn't rain today."
]

target_texts = [
  "What is your favorite food?",
  "Where do you live?",
  "Morning trains are crowded.",
  "It's nice weather today.",
  "The economy is bad lately"
]

in_embeds = embedding(in_texts)
target_embeds = embedding(target_texts)

In [14]:
import numpy as np

in_embeds = np.array(in_embeds).astype("float32")
target_embeds = np.array(target_embeds).astype("float32")

In [16]:
import faiss

index = faiss.IndexFlatL2(len(in_embeds[0]))
index.add(target_embeds)

distances, indices = index.search(in_embeds, 1)

print(distances)
print(indices)
print(target_texts[indices[0][0]])

[[0.5111765]]
[[3]]
It's nice weather today.


In [2]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)

def embedding(texts) :
  return model.encode(texts)["dense_vecs"]

texts = ["이것은 테스트입니다."]

embeds = embedding(texts)

print(embeds)
print(len(embeds[0]))

/opt/anaconda3/envs/ai-engineering/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [01:30<00:00,  3.02s/it]


In [3]:
in_texts = [
  "오늘은 비가 안 와서 다행입니다."
]

target_texts = [
  "가장 좋아하는 음식은 무엇인가요?",
  "어디에 사시나요?",
  "아침 기차는 붐빕니다.",
  "오늘은 날씨가 좋네요.",
  "최근 경기가 안 좋습니다."
]

in_embeds = embedding(in_texts)
target_embeds = embedding(target_texts)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [4]:
import numpy as np

in_embeds = np.array(in_embeds).astype("float32")
target_embeds = np.array(target_embeds).astype("float32")

In [8]:
import faiss

index = faiss.IndexFlatL2(len(in_embeds[0]))

index.add(target_embeds)

distances, indices = index.search(in_embeds, 1)

print(distances)
print(indices)
print(target_texts[indices[0][0]])

[[0.34107807]]
[[3]]
오늘은 날씨가 좋네요.
